In [ ]:
pip install openai pinecone-client langchain

In [ ]:
pip install -U langchain-community

In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key="pinecone_api_key")
index = pc.Index("langchain-chatbot")

In [ ]:
pip install -U langchain-openai

In [ ]:
from langchain_openai import OpenAIEmbeddings

# Example list of documents
documents = [
    "The quick brown fox jumps over the lazy dog.",
    "Artificial intelligence is transforming the world.",
    "Python is a versatile programming language used for various applications.",
    "Pinecone provides a scalable vector database for machine learning applications.",
    "OpenAI's GPT-4 model is capable of generating human-like text."
]

# Create embeddings
embeddings = OpenAIEmbeddings(api_key='api-key')
vectors = []

for i, doc in enumerate(documents):
    vector = embeddings.embed_query(doc)
    vectors.append((f'doc_{i}', vector, {'text': doc}))

# Upsert vectors into Pinecone
index.upsert(vectors)


In [ ]:
query = "User's question"
query_vector = embeddings.embed(query)

# Retrieve relevant documents
results = index.query(query_vector, top_k=5)
relevant_docs = [result['metadata']['text'] for result in results['matches']]


In [ ]:
import openai

context = " ".join(relevant_docs)
response = openai.Completion.create(
    engine="text-davinci-003",
    prompt=f"Answer the question based on the context: {context}\n\nQuestion: {query}\nAnswer:",
    max_tokens=150
)

print(response.choices[0].text.strip())


In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/ask', methods=['POST'])
def ask():
    data = request.json
    query = data['question']
    query_vector = embeddings.embed(query)
    results = index.query(query_vector, top_k=5)
    relevant_docs = [result['metadata']['text'] for result in results['matches']]
    context = " ".join(relevant_docs)
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=f"Answer the question based on the context: {context}\n\nQuestion: {query}\nAnswer:",
        max_tokens=150
    )
    return jsonify({'answer': response.choices[0].text.strip()})

if __name__ == '__main__':
    app.run(debug=True)
